In [8]:
import numpy as np
import collections
from math import log
import pandas as pd


In [9]:
class Node:
    def __init__(self):
        self.left = None
        self.right = None
        self.feature = None
        self.entropy = None
        self.values = None


In [10]:
def informationSplit(x_train,x1,x2):
    a = len(x1)
    b = len(x2)
    t = len(x_train)
    ans = (-(a/t)*log(a/t , 2)) - ((b/t)*log(b/t,2))
    return ans

In [11]:
def entropy(tmp):
    total = 0
    for key in tmp:
        total += tmp[key]
    ans = 0
    for key in tmp:
            temp = tmp[key] / total
            if(temp==0):
                continue
            ans = ans - (temp * log(temp,2))
    return ans

In [12]:
def gain(x_train,x1,x2,classes):
    if(len(x1)==0 or len(x2)==0):
        return -1
    x_train_classes = collections.Counter(x_train[:,len(x_train[0])-1])
    x1_classes = collections.Counter(x1[:,len(x1[0])-1])
    x2_classes = collections.Counter(x2[:,len(x2[0])-1])
    info_gain = entropy(x_train_classes) - ((len(x1)/len(x_train))*entropy(x1_classes) + (len(x2)/len(x_train))*entropy(x2_classes))
    gain_ratio = info_gain/informationSplit(x_train,x1,x2)
    return gain_ratio

In [13]:
def Split(x_train,features,classes):
    max_feature = 0
    max_gain= -1
    breakPoint = 0
    for f in features:
        arr = x_train[:,f]
        np.sort(arr)
        mid_list = []
        for i in range(len(arr)-1):
            temp = (arr[i] + arr[i+1])/2
            mid_list.append(temp)
        for i in range(len(mid_list)):
            condition = x_train[:,f] < mid_list[i]
            x1 = x_train[condition==True]
            x2 = x_train[condition==False]
            temp = gain(x_train,x1,x2,classes)
            if(temp>max_gain):
                max_gain = temp
                breakPoint = mid_list[i]
                max_feature = f
    return breakPoint,max_feature,max_gain

In [17]:
def build(x_train,features,classes,level):
    
    tmp = collections.Counter(x_train[:,len(x_train[0])-1])
    root = Node()
    root.entropy = entropy(tmp)
    root.values = tmp
    root.total_samples = len(x_train)
    print("Level",level)
    for i in classes:
        print("Count of ",i," = ",tmp[i])
    print("Current Entropy is ",root.entropy)
   
    
    if (len(features)==0):
        print("Reached Leaf Node")
        print()
        return None
    
    if(len(tmp) == 1):
        print("Reached Leaf Node")
        print()
        return root
    
    splitting, feature_used,gain = Split(x_train,features,classes)
    print("Splitting on feature",feature_used,"with gain ratio ",gain)
    
    index = features.index(feature_used)
    features.remove(feature_used)
    expr = x_train[:,feature_used] < splitting
    new_x_left = x_train[expr == True]
    new_x_right = x_train[expr == False]
    print()
    root.left = build(new_x_left,features,classes,level+1)
    root.right = build(new_x_right,features,classes,level+1)
    features.insert(index,feature_used)
    
    return root
    
    
    

In [15]:
def fit(data):
    x_train = data[:,:]
    features = [i for i in range(len(x_train[0])-1)]
    classes = collections.Counter(x_train[:,len(x_train[0])-1])
    
    tree = build(x_train,features,classes,0)
    return tree

In [18]:
data = np.array([[1,1,1],
                   [1,0,1],
                   [0,1,1],
                   [0,0,0]])
root = fit(data)
    

Level 0
Count of  1  =  3
Count of  0  =  1
Current Entropy is  0.8112781244591328
Splitting on feature 0 with gain ratio  0.31127812445913283

Level 1
Count of  1  =  1
Count of  0  =  1
Current Entropy is  1.0
Splitting on feature 1 with gain ratio  1.0

Level 2
Count of  1  =  0
Count of  0  =  1
Current Entropy is  0.0
Reached Leaf Node

Level 2
Count of  1  =  1
Count of  0  =  0
Current Entropy is  0.0
Reached Leaf Node

Level 1
Count of  1  =  2
Count of  0  =  0
Current Entropy is  0.0
Reached Leaf Node

